<h1 style="direction:rtl;"> מגישים:
</h1>
<h1 style='color:black; direction:rtl;'> אגת אהרפי 209155696 <br><br>
    נאור ברזילי 313153439
    </h1>


In [ ]:
<h3 style='color:black'>Preparing the data</h3>


In [107]:
import pandas as pd
import nltk
import math
import re 
import collections
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('names')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist


df = pd.read_csv('Twitter_data.csv')
df

[nltk_data] Downloading package words to C:\Users\אגת
[nltk_data]     אהרפי\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\אגת
[nltk_data]     אהרפי\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\אגת
[nltk_data]     אהרפי\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package names to C:\Users\אגת
[nltk_data]     אהרפי\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\אגת
[nltk_data]     אהרפי\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,twitter_screen_name,twitter_name,twitter_desc,twitter_location,wikidata_desc,type1,type2
0,BarackObama,Barack Obama,"Dad, husband, President, citizen.","Washington, DC",44th president of the United States,Politician,Person
1,justinbieber,Justin Bieber,#Changes out now,NaN,Canadian singer,MusicalArtist,Person
2,katyperry,KATY PERRY,Love. Light.,NaN,American singer,MusicalArtist,Person
3,rihanna,Rihanna,NaN,NaN,"Barbadian singer, songwriter, and businesswoman",MusicalArtist,Person
4,taylorswift13,Taylor Swift,Lover out now,NaN,American singer-songwriter,MusicalArtist,Person
...,...,...,...,...,...,...,...
29058,JessalynGilsig,Jessalyn Gilsig,Big Shot Vikings Glee Scandal Nip Tuck and any...,NaN,actress,Person,Person
29059,ReemaMajor,Reema Major,Dey Say Only Da Good Die Young So Fuck It ima ...,South Sudan - UAE,Canadian rapper,MusicalArtist,Person
29060,EqualityTexas,Equality Texas,We work to secure full equality for LGBTQ Texa...,"Austin, Texas",NaN,Organisation,Organisation
29061,Margus_Hunt,Margus Hunt,Official Twitter page of Margus Hunt,"Indianapolis, IN","Estonian track and field athlete, American foo...",AmericanFootballPlayer,Person


In [108]:
# drop duplicates (keeping only the first)
df = df.drop_duplicates(subset='twitter_screen_name', keep='first')

df

,twitter_screen_name,twitter_name,twitter_desc,twitter_location,wikidata_desc,type1,type2
0,BarackObama,Barack Obama,"Dad, husband, President, citizen.","Washington, DC",44th president of the United States,Politician,Person
1,justinbieber,Justin Bieber,#Changes out now,NaN,Canadian singer,MusicalArtist,Person
2,katyperry,KATY PERRY,Love. Light.,NaN,American singer,MusicalArtist,Person
3,rihanna,Rihanna,NaN,NaN,"Barbadian singer, songwriter, and businesswoman",MusicalArtist,Person
4,taylorswift13,Taylor Swift,Lover out now,NaN,American singer-songwriter,MusicalArtist,Person
...,...,...,...,...,...,...,...
29058,JessalynGilsig,Jessalyn Gilsig,Big Shot Vikings Glee Scandal Nip Tuck and any...,NaN,actress,Person,Person
29059,ReemaMajor,Reema Major,Dey Say Only Da Good Die Young So Fuck It ima ...,South Sudan - UAE,Canadian rapper,MusicalArtist,Person
29060,EqualityTexas,Equality Texas,We work to secure full equality for LGBTQ Texa...,"Austin, Texas",NaN,Organisation,Organisation
29061,Margus_Hunt,Margus Hunt,Official Twitter page of Margus Hunt,"Indianapolis, IN","Estonian track and field athlete, American foo...",AmericanFootballPlayer,Person


<h3 style='color:black'>Helper Functions</h3>

In [ ]:
# gets a list of sentences, splits to words and returns a list of them
def list_splited_words(listWords):
    english_vocab = sorted(set(w.lower() for w in nltk.corpus.words.words()))
    stopwords = nltk.corpus.stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    splited =[]
    for sen in listWords:
        if(isinstance(sen, str)):  # only if not NaN
            for word in nltk.word_tokenize(sen):
                word=word.lower()
                if word not in stopwords and word in english_vocab:
                    splited.append(lemmatizer.lemmatize(word))  
    return splited

#-------------------------------------------------------------------------
# calculates PMI
def PMI(class_size, other_class_size, class_dataset, other_class_dataset):    
    pmi_list=[]
    pr_class= class_size / (class_size + other_class_size)
    for word in set(class_dataset):
        pr_w_class = class_dataset.count(word) / len(class_dataset)
        pr_w = (other_class_dataset.count(word) + class_dataset.count(word)) / (len(class_dataset) + len(other_class_dataset))
        pmi = math.log2(pr_w_class / (pr_w * pr_class))
        pmi_list.append((word,pmi))
    sorted_by_pmi = sorted(pmi_list, key=lambda tup: tup[1], reverse=True)   
    return sorted_by_pmi

#------------------------------------------------------------------------
#check the sentence's sentiment (positive/negative)
def check_sentiment(sen):
    count=0
    lemmatizer = WordNetLemmatizer()
    for word in nltk.word_tokenize(sen):
        word=lemmatizer.lemmatize(word)
        if(word in positive_words):
            count+=1
        if(word in negative_words):
            count-=1
    return count # sentiment determined by the amount of positive/negative words

<h1 style='color:red'>Question 1:</h1>


In [111]:
english_vocab = sorted(set(w.lower() for w in nltk.corpus.words.words()))
stopwords = nltk.corpus.stopwords.words('english')

col_one_list = df['twitter_desc'].tolist() # putting all the twitter_desc into a list
lemmatizer = WordNetLemmatizer()
splited =[]
for sen in col_one_list:
    if(isinstance(sen, str)): # only if not NaN
        for word in nltk.word_tokenize(sen):
            word=word.lower()
            if word not in stopwords and word in english_vocab:
                splited.append(lemmatizer.lemmatize(word))  
fdist1=FreqDist(splited) # make a list of freq
print(fdist1.most_common(30)) # print the 30 most frequent 

[('news', 1152), ('account', 1150), ('official', 1023), ('world', 898), ('de', 817), ('u', 713), ('life', 483), ('new', 454), ('author', 450), ('people', 446), ('book', 437), ('since', 399), ('help', 379), ('page', 372), ('make', 359), ('love', 357), ('album', 351), ('music', 341), ('best', 310), ('thing', 309), ('la', 305), ('available', 295), ('writer', 294), ('time', 293), ('tweet', 292), ('business', 292), ('home', 290), ('one', 287), ('sport', 279), ('year', 279)]


<h1 style='color:red'>Question 2:</h1>

In [113]:
names = nltk.corpus.names
names.fileids()
male_names = names.words('male.txt')
female_names = names.words('female.txt')

names_and_desc = []

# create a tuple of (name, twitter_desc)
for row in df.itertuples():
    if(isinstance(row[2], str)):
        name=re.sub('\W+','-', row[2]) # (row[2] - full name)
        names_and_desc.append((name.split('-')[0].capitalize(),row[3]))  # split the first and last name 
        
male_list_name=[]
female_list_name=[]

male_words=[]
female_words=[]
# for each name add the name and desc for the correct list(male/female)
for tup in names_and_desc:
    if tup[0] in male_names and tup[0] not in female_names:
        male_list_name.append(name)
        male_words.append(tup[1])
    if tup[0] in female_names and tup[0] not in male_names:
        female_list_name.append(name)
        female_words.append(tup[1])

print("Amount of female users: " + str(len(female_list_name)))
print("Amount of male users: " + str(len(male_list_name)))


Amount of female users: 3558
Amount of male users: 6370


In [114]:
print("Female most common:")
splited_female=list_splited_words(female_words)
fdist1=FreqDist(splited_female)
print(fdist1.most_common(30))

Female most common:
[('author', 328), ('actress', 249), ('new', 197), ('official', 191), ('twitter', 160), ('writer', 159), ('wife', 143), ('world', 136), ('host', 126), ('former', 126), ('lover', 121), ('actor', 116), ('de', 115), ('time', 114), ('mother', 112), ('love', 107), ('la', 100), ('book', 93), ('life', 91), ('news', 89), ('founder', 84), ('singer', 79), ('director', 79), ('girl', 78), ('account', 78), ('producer', 73), ('champion', 72), ('speaker', 66), ('contact', 64), ('u', 62)]


In [115]:
print("Male most common:")
splited_male=list_splited_words(male_words)
fdist1=FreqDist(list_splited_words(splited_male))
print(fdist1.most_common(30))


Male most common:
[('official', 512), ('author', 453), ('twitter', 440), ('new', 363), ('husband', 298), ('father', 289), ('former', 281), ('actor', 280), ('world', 235), ('host', 213), ('account', 212), ('de', 203), ('writer', 200), ('time', 187), ('news', 172), ('champion', 163), ('player', 160), ('director', 154), ('dad', 147), ('founder', 146), ('life', 142), ('music', 140), ('love', 128), ('book', 127), ('page', 123), ('professional', 117), ('producer', 117), ('u', 115), ('la', 113), ('university', 113)]


In [116]:
# CHECK!!
print("PMI MALE: ")
print("splited_male")
print(splited_male)
print("splited_female")
print(splited_female)


PMI MALE: 
splited_male
['lover', 'president', 'united', 'astrophysicist', 'film', 'actor', 'artist', 'painter', 'humanitarian', 'new', 'album', 'heartbreak', 'weather', 'name', 'kevin', 'hart', 'work', 'hard', 'pretty', 'much', 'everybody', 'famous', 'nobody', 'work', 'one', 'direction', 'would', 'nowhere', 'without', 'incredible', 'owe', 'debut', 'album', 'listen', 'actor', 'david', 'football', 'player', 'president', 'founder', 'actor', 'environmentalist', 'guy', 'follow', 'america', 'cable', 'news', 'network', 'breaking', 'news', 'insightful', 'analysis', 'use', 'please', 'follow', 'lot', 'love', 'x', 'official', 'social', 'medium', 'account', 'actor', 'jim', 'del', 'real', 'madrid', 'chill', 'build', 'believe', 'future', 'right', 'pursue', 'happiness', 'del', 'de', 'la', 'de', 'actor', 'sometimes', 'pretty', 'good', 'shape', 'time', 'hey', 'got', 'ta', 'live', 'know', 'barcelona', 'player', 'born', 'january', 'official', 'twitter', 'account', 'world', 'leading', 'voice', 'entrepren

In [117]:
#--------------PMI Male----------------
male_pmi=PMI(len(male_list_name), len(female_list_name), splited_male, splited_female)
highest_pmi_male = male_pmi[0:30]
highest_pmi_male

[('wednesday', 1.2824596202239196),
 ('palmer', 1.2824596202239196),
 ('viking', 1.2824596202239196),
 ('course', 1.2824596202239196),
 ('poor', 1.2824596202239196),
 ('rogue', 1.2824596202239196),
 ('river', 1.2824596202239196),
 ('madrid', 1.2824596202239196),
 ('racer', 1.2824596202239196),
 ('castle', 1.2824596202239196),
 ('richard', 1.2824596202239196),
 ('architect', 1.2824596202239196),
 ('silicon', 1.2824596202239196),
 ('austin', 1.2824596202239196),
 ('hotspur', 1.2824596202239196),
 ('memory', 1.2824596202239196),
 ('chuck', 1.2824596202239196),
 ('wisdom', 1.2824596202239196),
 ('cash', 1.2824596202239196),
 ('roger', 1.2824596202239196),
 ('subcommittee', 1.2824596202239196),
 ('offering', 1.2824596202239196),
 ('quarterback', 1.2824596202239196),
 ('cable', 1.2824596202239196),
 ('cast', 1.2824596202239196),
 ('richmond', 1.2824596202239196),
 ('constitution', 1.2824596202239196),
 ('tight', 1.2824596202239196),
 ('coverage', 1.2824596202239196),
 ('evangelist', 1.282459

In [118]:
#--------------PMI Female----------------
female_pmi=PMI(len(female_list_name), len(male_list_name), splited_female, splited_male)
highest_pmi_female = female_pmi[0:30]
highest_pmi_female

[('disabled', 2.957227883424215),
 ('congresswoman', 2.957227883424215),
 ('marie', 2.957227883424215),
 ('pronoun', 2.957227883424215),
 ('dare', 2.957227883424215),
 ('wander', 2.957227883424215),
 ('katy', 2.957227883424215),
 ('nurse', 2.957227883424215),
 ('nap', 2.957227883424215),
 ('baking', 2.957227883424215),
 ('mermaid', 2.957227883424215),
 ('fit', 2.957227883424215),
 ('meal', 2.957227883424215),
 ('bird', 2.957227883424215),
 ('decision', 2.957227883424215),
 ('ava', 2.957227883424215),
 ('soprano', 2.957227883424215),
 ('reading', 2.957227883424215),
 ('laurel', 2.957227883424215),
 ('pray', 2.957227883424215),
 ('samantha', 2.957227883424215),
 ('normal', 2.957227883424215),
 ('jennifer', 2.957227883424215),
 ('bloom', 2.957227883424215),
 ('brat', 2.957227883424215),
 ('amber', 2.957227883424215),
 ('mummy', 2.957227883424215),
 ('bittersweet', 2.957227883424215),
 ('bitch', 2.957227883424215),
 ('medal', 2.957227883424215)]

<h1 style='color:red'>Question 3:</h1>
<h3 style='color:red'>Section A:</h3>

In [ ]:
musical_artist_words=[] # will contain a list of words of musicians 
not_musical_artist_words=[] # will contain a list of words of non musicians 
count_musicians=0 # amount of musical artists on the dataset
count_non_musicians=0 # amount of the rest

for row in df.itertuples():
    if(isinstance(row[6], str) and row[6]=='MusicalArtist'): # row[6] - type1
        musical_artist_words.append(row[3]) # add the twitter_desc to the list
        count_musicians+=1
    else:
        not_musical_artist_words.append(row[3]) # add the twitter_desc to the list
        count_non_musicians+=1
        
# split the twitter_desc to a list of words        
musical_artist_words=list_splited_words(musical_artist_words) 
not_musical_artist_words=list_splited_words(not_musical_artist_words)

#---------PMI MUSICIANS--------
musical_pmi=PMI(count_musicians, count_non_musicians, musical_artist_words, not_musical_artist_words)
highest_pmi_musicians = musical_pmi[0:30]
highest_pmi_musicians


<h3 style='color:red'>Section B:</h3>

In [ ]:
political_words=[]  # will contain a list of words of politicans 
not_political_words=[] # will contain a list of words of non politicans
count_political=0 # amount of politicans on the dataset
count_non_political=0 # amount of the rest

for row in df.itertuples():
    if(isinstance(row[6], str) and row[6]=='Politician'): # row[6] - type1
        political_words.append(row[3]) # add the twitter_desc to the list
        count_political+=1
    else:
        not_political_words.append(row[3]) # add the twitter_desc to the list
        count_non_political+=1

# split the twitter_desc to a list of words
political_words=list_splited_words(political_words)
not_political_words=list_splited_words(not_political_words)

#---------PMI POLITICANS--------
political_pmi=PMI(count_political, count_non_political, political_words, not_political_words)
highest_pmi_political = political_pmi[0:30]
highest_pmi_political

<h1 style='color:red'>Question 4:</h1>
    *the lists of positive and negative words were taken from: http://www.cs.uic.edu/~liub/FBS/opinion-lexicon-English.rar
    (we took it from Subjectivity and Sentiment Analysis chapter)    

In [ ]:
positive_file = open("positive-words.txt", "r") # read the file of positive words 
  
# reading the file
data = positive_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
positive_words = data.split("\n")
positive_file.close()

negative_file = open("negative-words.txt", "r") # read the file of negative words 
  
# reading the file
data = negative_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
negative_words = data.split("\n")
negative_file.close()

In [ ]:
col_one_list = df['twitter_desc'].tolist() # create a list of twitter_desc 
rank_list = [] # tuple of (sentence, rank(positive/negative))

for sen in col_one_list:
    if(isinstance(sen, str)):
        rank_list.append((sen, check_sentiment(sen)))
        
sorted_rank_list = sorted(rank_list, key=lambda tup: tup[1], reverse=True) # sort - most positive first
positive_sorted_rank_list = sorted_rank_list[0:10]
print("most positive sen:\n")
print(positive_sorted_rank_list)

sorted_rank_list = sorted(rank_list, key=lambda tup: tup[1], reverse=False)  # sort - most negative first
negative_sorted_rank_list = sorted_rank_list[0:10]
print("most negative sen:\n")
print(negative_sorted_rank_list)

<h1 style='color:red'>Question 5:</h1>

In [ ]:
col_one_list = df['twitter_desc'].tolist() # create a list of twitter_desc
count_hebrew=0 # amount of sentences that contain at least one char in hebrew
count_arabic=0 # amount of sentences that contain at least one char in arabic

for sen in col_one_list:
    if(isinstance(sen,str) and re.search(r'[\u0600-\u06FF\u0750-\u077F]+', sen) is not None): # if contains an arabic char
        count_arabic+=1
    if(isinstance(sen,str) and re.search(r"[א-ת]+", sen) is not None): # if contains a hebrew char
        count_hebrew+=1
        
hebrew_percentage = count_hebrew / len(col_one_list)
print("hebrew: "+ str(count_hebrew) + "   " + str(hebrew_percentage)+"%")

arabic_percentage = count_arabic / len(col_one_list)
print("arabic: "+str(count_arabic)+ "   "+ str(arabic_percentage)+"%")


<h1 style='color:red'>Question 6:</h1>

In [92]:
# create a string that contains all the emoticons in the twitter_desc 
desc_string = "" 
col_one_list = df['twitter_desc'].tolist() # create a list of twitter_desc 
for sen in col_one_list:
    if(isinstance(sen,str)): # only if not NaN
        for char in re.findall(r'[\U0001f600-\U0001f650]+',sen): # return only emoticons 
            desc_string=desc_string + str(char)

emoji_counter = collections.Counter(desc_string) # count the occurrence of each emoticon 
emoji_counter.most_common(10)


error: incomplete escape \x at position 2